In [1]:
# Standard libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Handeling .wav files

import librosa
from librosa import feature

# Machine Learning

from sklearn.pipeline import Pipeline

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, RepeatedStratifiedKFold

from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from functools import reduce

# data vizualisation

import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def merge_pd(machine):
    
    # Opening the .csv files
    
    df_6dB=pd.read_csv(f'Librosa_features_{machine}_6dB.csv')
    df_0dB=pd.read_csv(f'Librosa_features_{machine}_0dB.csv')
    df_min6dB=pd.read_csv(f'Librosa_features_{machine}_-6dB.csv')
    
    # Dropping the 'Unnamed: 0' column
    
    df_6dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    df_0dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    df_min6dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    

    
    # Merging the .csv files into one DataFrame
    
    data_frames = [df_6dB, df_0dB, df_min6dB]
    df_merged = pd.concat(data_frames)
    
    
    return df_merged

In [3]:
valve = merge_pd('valve')

In [12]:
valve.shape

(12510, 21)

In [6]:
minority_class_len = len(valve[valve['normal(0)/abnormal(1)'] == 1])
majority_class_indices = valve[valve['normal(0)/abnormal(1)'] == 0].index
random_majority_indices = np.random.choice(majority_class_indices, minority_class_len , replace= False)
minority_class_indices  = valve[valve['normal(0)/abnormal(1)'] == 1].index

under_sample_indices = np.concatenate( [minority_class_indices , random_majority_indices])
under_sample = valve.loc[under_sample_indices]
X = under_sample.drop(columns = ['normal(0)/abnormal(1)'])
y = under_sample['normal(0)/abnormal(1)']

# 20% of the overal data will seperated for later validation of the model

X_model, X_valid, y_model, y_valid = train_test_split(X, y, test_size=0.2, random_state = 42, stratify = y)

# 60% of the overal data (75% of X_model, y_model) will be used to create a training set for the model
# 20% of the overal data (25% of X_model, y_model) will be used to create a testing set for the model

X_train, X_test, y_train, y_test = train_test_split(X_model,
                                                   y_model,
                                                   test_size = 0.25,
                                                   random_state = 10, stratify=y_model)

In [9]:
print(X_train.shape, y_train.shape,X_test.shape, y_test.shape,X_valid.shape, y_valid.shape)
print(X,y)

(5172, 20) (5172,) (1725, 20) (1725,) (1725, 20) (1725,)
      melspectrogram  melspectrogram_min  melspectrogram_max  \
3691        0.005408        6.000231e-10            0.041900   
3691        0.011661        1.195045e-09            0.195286   
3691        0.021925        2.777673e-09            0.420920   
3692        0.005095        6.719894e-10            0.124696   
3692        0.009716        1.121630e-09            0.196361   
...              ...                 ...                 ...   
350         0.009826        1.072683e-09            0.284180   
350         0.022400        2.481091e-09            0.342450   
3018        0.006673        7.509776e-10            0.287420   
3018        0.010661        1.199072e-09            0.357471   
3018        0.022924        2.796089e-09            0.321063   

      melspectrogram_sum  melspectrogram_corr  melspectrogram_std       mfcc  \
3691          298.331635             0.127720            0.004372 -17.873192   
3691          

In [10]:
pipeline = Pipeline([('scaler', StandardScaler()), ('clf', SVC())])

In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 56em; }</style>"))

In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.feature_selection import RFE

y_pred = []
clfs = []

clfs.append(DecisionTreeClassifier())
clfs.append(RandomForestClassifier())
#clfs.append(EasyEnsembleClassifier(importance_getter=index))

#feature_list = [6,13,21]

#for index in feature_list:
for index in range(4,20):
    
    for classifier in clfs:
        
        print("----------------------------------------------")
        print("----------------------------------------------")
        print(classifier)
        print("----------------------------------------------")
        print("----------------------------------------------")
        
        sel = RFE(classifier, n_features_to_select = index)
        sel.fit(X_train, y_train)
        features = X_train.columns[sel.get_support()]
        X_train_rfe = sel.transform(X_train)
        X_test_rfe = sel.transform(X_test)
        print('Selected Feature', index)
        print(features)
    
        classifier.fit(X_train_rfe, y_train)
        y_pred= classifier.predict(X_test_rfe)
        scores = cross_val_score(pipeline, X_train_rfe, y_train, cv=5)


        print("----------------------------------------------")
        print("TRAIN-TEST")
        print("----------------------------------------------")


        print('confusion matrix', classifier)
        print(confusion_matrix(y_test, y_pred))
        print('classification report')
        print(classification_report(y_test, y_pred))
        print('accuracy score')
        print(accuracy_score(y_test, y_pred))
        
        X_valid_rfe = sel.transform(X_valid)
        y_pred = classifier.predict(X_valid_rfe)

        print("----------------------------------------------")
        print("TRAIN-VALIDATION")
        print("----------------------------------------------")

        print('confusion matrix', classifier)
        print(confusion_matrix(y_valid, y_pred))
        print('classification report')
        print(classification_report(y_valid, y_pred))
        print('accuracy score')
        print(accuracy_score(y_valid, y_pred))

----------------------------------------------
----------------------------------------------
DecisionTreeClassifier()
----------------------------------------------
----------------------------------------------
Selected Feature 4
Index(['melspectrogram_max', 'melspectrogram_corr', 'max perc', 'min perc'], dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[757 105]
 [ 56 807]]
classification report
              precision    recall  f1-score   support

           0       0.93      0.88      0.90       862
           1       0.88      0.94      0.91       863

    accuracy                           0.91      1725
   macro avg       0.91      0.91      0.91      1725
weighted avg       0.91      0.91      0.91      1725

accuracy score
0.9066666666666666
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion m

Selected Feature 7
Index(['melspectrogram', 'melspectrogram_max', 'melspectrogram_corr',
       'melspectrogram_std', 'spectral_rolloff', 'max perc', 'min perc'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[752 110]
 [ 26 837]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.87      0.92       862
           1       0.88      0.97      0.92       863

    accuracy                           0.92      1725
   macro avg       0.93      0.92      0.92      1725
weighted avg       0.93      0.92      0.92      1725

accuracy score
0.9211594202898551
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[740 123]
 [ 28 834]]
classification report
              precision    recall  f1-score   support

         

Selected Feature 10
Index(['melspectrogram_max', 'melspectrogram_sum', 'melspectrogram_corr',
       'melspectrogram_std', 'mfcc', 'spectral_bandwidth',
       'spectral_flatness ', 'max perc', 'min harm', 'min perc'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[750 112]
 [ 36 827]]
classification report
              precision    recall  f1-score   support

           0       0.95      0.87      0.91       862
           1       0.88      0.96      0.92       863

    accuracy                           0.91      1725
   macro avg       0.92      0.91      0.91      1725
weighted avg       0.92      0.91      0.91      1725

accuracy score
0.9142028985507247
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[739 124]
 [ 36 826]]
classification report
        

Selected Feature 13
Index(['melspectrogram_max', 'melspectrogram_sum', 'melspectrogram_corr',
       'melspectrogram_std', 'mfcc', 'rms', 'spectral_bandwidth',
       'spectral_contrast', 'spectral_flatness ', 'zero_crossing_rate',
       'max perc', 'min harm', 'min perc'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[748 114]
 [ 31 832]]
classification report
              precision    recall  f1-score   support

           0       0.96      0.87      0.91       862
           1       0.88      0.96      0.92       863

    accuracy                           0.92      1725
   macro avg       0.92      0.92      0.92      1725
weighted avg       0.92      0.92      0.92      1725

accuracy score
0.9159420289855073
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifie

----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier()
[[810  52]
 [ 35 828]]
classification report
              precision    recall  f1-score   support

           0       0.96      0.94      0.95       862
           1       0.94      0.96      0.95       863

    accuracy                           0.95      1725
   macro avg       0.95      0.95      0.95      1725
weighted avg       0.95      0.95      0.95      1725

accuracy score
0.9495652173913044
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix RandomForestClassifier()
[[814  49]
 [ 28 834]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.94      0.95       863
           1       0.94      0.97      0.96       862

    accuracy                           0.96      1725
   macro avg       0.96      0.

----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[751 111]
 [ 29 834]]
classification report
              precision    recall  f1-score   support

           0       0.96      0.87      0.91       862
           1       0.88      0.97      0.92       863

    accuracy                           0.92      1725
   macro avg       0.92      0.92      0.92      1725
weighted avg       0.92      0.92      0.92      1725

accuracy score
0.9188405797101449
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[754 109]
 [ 31 831]]
classification report
              precision    recall  f1-score   support

           0       0.96      0.87      0.92       863
           1       0.88      0.96      0.92       862

    accuracy                           0.92      1725
   macro avg       0.92      0.